In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle

# set warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# add parent folder path to the namespace
sys.path.append(os.path.dirname(os.getcwd()))

# import modules and components
from utils.preprocessing import PreProcessing
from utils.inference import Inference
from utils.validation import ModelValidation
import utils.global_paths as globpt
import configurations as cnf

# specify relative paths from global paths and create subfolders
cp_path = os.path.join(globpt.train_path, 'checkpoints')
os.mkdir(cp_path) if not os.path.exists(cp_path) else None

# Load data and model

In [3]:
# load the model for inference and print summary
inference = Inference(cnf.seed) 
model, parameters = inference.load_pretrained_model(cp_path)
model_path = inference.folder_path
model.summary(expand_nested=True)

# load preprocessed data
pp_path = os.path.join(model_path, 'preprocessing')

# define preprocessed data path
preprocessing_path = os.path.join(model_path, 'preprocessing')

# load preprocessed csv files (train and test datasets)
file_loc = os.path.join(preprocessing_path, 'X_train.csv') 
train_X = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'X_test.csv') 
test_X = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'Y_train.csv') 
train_Y = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'Y_test.csv') 
test_Y = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)

# split sequences into items (from string)
#------------------------------------------------------------------------------ 
train_X[''] = train_X[]


# generate tf.datasets
#------------------------------------------------------------------------------ 
preprocessor = PreProcessing()
train_dataset, test_dataset = preprocessor.create_tf_dataset(train_X, test_X,
                                                             train_Y, test_Y,
                                                             parameters['sequence_length'],
                                                             cnf.batch_size)

# load encoders and normalizers
filepath = os.path.join(pp_path, 'pressure_normalizer.pkl')
with open(filepath, 'rb') as file:
    press_normalizer = pickle.load(file)
filepath = os.path.join(pp_path, 'uptake_normalizer.pkl')
with open(filepath, 'rb') as file:
    uptake_normalizer = pickle.load(file)

Model: "SCADS"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 40)]         0           []                               
                                                                                              

ValueError: cannot reshape array of size 1 into shape (40,)

# Model evaluation

### Evaluation of loss and metrics

In [ ]:
# initialize custom classes
validator = ModelValidation(model)

# create subfolder for evaluation data
eval_path = os.path.join(model_path, 'evaluation') 
os.mkdir(eval_path) if not os.path.exists(eval_path) else None

# evaluate model performance on train and test datasets
train_eval = model.evaluate(x=train_inputs, y=train_uptakes, batch_size=512, verbose=1)
test_eval = model.evaluate(x=test_inputs, y=test_uptakes, batch_size=512, verbose=1)

print(f'''
-------------------------------------------------------------------------------
MODEL EVALUATION
-------------------------------------------------------------------------------    
Train dataset:
- Loss:   {train_eval[0]}
- Metric: {train_eval[1]} 

Test dataset:
- Loss:   {test_eval[0]}
- Metric: {test_eval[1]}        
''')

### Visual inspection

In [ ]:
# predict adsorption from train and test datasets input
train_predictions = model.predict(train_inputs)
test_predictions = model.predict(test_inputs)

# remove padding and normalization from the original train and test pressure series,
# as well from the original train and test uptake series and the predicted values
rec_train_P, rec_train_Q, pred_train_Q = inference.sequence_recovery(train_pressures,                                                                     
                                                                     train_uptakes,
                                                                     train_predictions,
                                                                     parameters['padding_value'],
                                                                     press_normalizer,
                                                                     uptake_normalizer)  

rec_test_P, rec_test_Q, pred_test_Q = inference.sequence_recovery(test_pressures,
                                                                  test_uptakes,
                                                                  test_predictions,                                                                  
                                                                  parameters['padding_value'],
                                                                  press_normalizer,
                                                                  uptake_normalizer)  

# perform visual validation by comparing true and predicted isotherms on both 
# the train and test datasets
validator.visualize_predictions(rec_train_P, rec_train_Q, pred_train_Q, 'train', eval_path)
validator.visualize_predictions(rec_test_P, rec_test_Q, pred_test_Q, 'test', eval_path)
